In [ ]:
# Parameters
num_periods = 10
x = 1

In [1]:
from input_properties import Mine_properties, concentration_func
from mining_data import mining_data
from Mine import Mine
from Time import Time
from Scheduler import Scheduler
import pandas as pd
import json
import matplotlib.pyplot as plt

In [2]:
import requests
import json

# Ollama server URL
OLLAMA_URL = "http://localhost:11434/api/chat"
MODEL = "gpt-oss"

# Function to send messages to Ollama
def ask_ollama(messages, stream=False):
    payload = {
        "model": MODEL,
        "messages": messages,
        "stream": stream
    }
    resp = requests.post(OLLAMA_URL, json=payload)
    return resp.json()

# Conversation memory
conversation = []
gpt_connversation = []

In [ ]:
# Initialize mining data
# num_periods = 20
# x=3
state = mining_data((x, 2, 1))
arr, block_properties, HnD = state.build(concentration_func=concentration_func)

In [4]:
# # Create mine instance
# import random
# random.seed(42)
# print(arr.flatten())
# mine_tonnage_list = []
# for i in range(1,arr.flatten().shape[0]+1):
#     random_value = random.randint(50, 90)
#     mine_tonnage_list.append(random_value)

# Mine_properties['block_tonnage'] = mine_tonnage_list
mining_instance = Mine(
    arr,
    block_properties,
    HnD,
    num_periods,
    Mine_properties
)

In [5]:
time_func = Time()
choice = 'greedy'  # or 'minimum'
scheduler = Scheduler(mining_instance, time_func)

In [6]:
action_list = scheduler.possible_actions()
period = scheduler.time_func.period()
scheduler.update_action(action_list,action='initialize')

In [7]:
system_prompt = '''
You are a Mine Scheduler. I will provide you with a list of possible actions which are formatted as follow(choice, block_number, ton_present, per unit change in NPV if selected). If choice=1, it will do extraction if choice=2 it will do processing. If processing you will receive revenue and if extraction you will have coist. Your task is to chose one action such as(choice,block_number,amount_you_want_to_be_operated_on) which is optimal based on maximizing the Net Present Value (NPV) over a specified number of periods. Then I will give you the updated state after your action. You will repeat this process until all periods are completed. Keep in mind the following constraints:
1. After full extraction of block you will see that it can also be processed in the next action.
2. Extraction can cost but processing generates revenue.
3. Extraction cannot be 0 amount if possible.
4. The period will change when the limit exceeds, will know as works are being done.
Always respond in JSON format not in plain text. The format is as follow:
{
  "choice": [1/2],
  "block_number": (int),
  "amount": (int)
}
Always follow the format.
'''
conversation.append({"role": "system", "content": system_prompt})
gpt_connversation.append({"role": "system", "content": system_prompt})

In [8]:
prompt = f"Time_period-{period}: Here are the possible actions: {scheduler.possible_actions()}"
conversation.append({"role": "user", "content": prompt})
gpt_connversation.append({"role": "user", "content": prompt})

response = ask_ollama(gpt_connversation)

conversation.append({"role": "Scheduler", "content": response['message']})

In [9]:
# response['message']['content']

In [ ]:
while period <= num_periods:
    assistant_msg = response['message']['content']
    # Remove ```json and ``` if present
    clean = assistant_msg.strip().strip("`").replace("```json", "").replace("```", "").replace("json","").strip()
    # print("Cleaned Assistant Message:", clean)
    try:
        parsed = json.loads(clean)
    except json.JSONDecodeError as e:
        print("JSON decoding error:", e)
        print("Original Assistant Message:", assistant_msg)
        response = ask_ollama(gpt_connversation)
        continue
    # parsed = json.loads(assistant_msg)
    try:
        mapping = {
            "extract": 1,
            "extraction":1,
            "process": 2,
            "1": 1,
            "2": 2
        }
        raw_choice = parsed["choice"]

        if isinstance(raw_choice, int) and raw_choice in (1, 2):
            choice = raw_choice
        elif isinstance(raw_choice, str) and raw_choice.strip().lower() in mapping:
            choice = mapping[raw_choice.strip().lower()]
        else:
            raise ValueError("Invalid choice input.")
        
        raw_block = parsed.get("block_number", parsed.get("block"))
        if raw_block is None:
            raise KeyError("Neither 'block_number' nor 'block' found.")

        block_number = int(raw_block)
        
        amount = int(parsed['amount'])

    except KeyError as e:
        print("Key error:", e)
        print("Parsed JSON:", parsed)
        response = ask_ollama(gpt_connversation)
        continue
    except TypeError as e:
        print("Type error:", e)
        print("Parsed JSON:", parsed)
        response = ask_ollama(gpt_connversation)
        continue
    except ValueError as e:
        print("Value error:", e)
        print("Parsed JSON:", parsed)
        response = ask_ollama(gpt_connversation)
        continue
    
    print("Choice:", choice)
    print("Block Number:", block_number)
    print("Amount:", amount)
    action = (choice,block_number,amount)
    listed = scheduler.possible_actions()
    match_found = any(
                    c == action[0] and b == action[1] and action[2] <= cap 
                    for c, b, cap, _ in listed
                )

    if not match_found:
        response = ask_ollama(gpt_connversation)
        continue

    action, pv = scheduler.choosen_action(action)
    scheduler.update_action(action,present_value=pv)
    scheduler.action(action)
    if len(scheduler.possible_actions()) == 0:
        scheduler.time_tick()
        if len(scheduler.possible_actions()) == 0:
            print("No more possible actions available. Exiting loop.")
            break
        else:
            scheduler.update_action(scheduler.possible_actions(),action='initialize')
            
    prompt = f"choice was: {choice}, block_number was: {block_number}, amount was: {amount}. Current NPV is: {scheduler.mine.npv} Time_period-{period}: Here are the updated possible actions: {scheduler.possible_actions()}. Choose from possible actions on in this format('choice': [1/2],  'block_number':(int), 'amount': (int))"
    
    conversation.append({"role": "user", "content": prompt})
    gpt_connversation.append({"role": "user", "content": prompt})
    
    response = ask_ollama(gpt_connversation)
    # print(prompt)
    print("Scheduler Response:", response)
    conversation.append({"role": "Scheduler", "content": response['message']})
    period = scheduler.time_func.period()

Choice: 1
Block Number: 1
Amount: 1
Scheduler Response: {'model': 'gpt-oss', 'created_at': '2025-12-15T23:22:12.315754514Z', 'message': {'role': 'assistant', 'content': '{"choice":1,"block_number":1,"amount":1}', 'thinking': "We need to choose an action that maximizes NPV over periods. Currently only extraction actions given, all have cost -9.09. No processing available. So best is to extract 1 unit from block 1? But extraction cannot be 0. All blocks have 50 units. Should choose block that gives best future processing? But all same. Probably choose block 1 again? But might want to reduce block 1 to 49. It's the same cost. Could choose any. Might as well choose block 1 again.\n\nThus output JSON."}, 'done': True, 'done_reason': 'stop', 'total_duration': 2529142463, 'load_duration': 168580099, 'prompt_eval_count': 662, 'prompt_eval_duration': 93100585, 'eval_count': 137, 'eval_duration': 2208575984}
Choice: 1
Block Number: 1
Amount: 1
Scheduler Response: {'model': 'gpt-oss', 'created_at

In [ ]:
scheduler.possible_actions()

In [ ]:
gpt_connversation

In [ ]:
scheduler.time_func.period()

In [ ]:
scheduler.mine.ore_tonnage

In [ ]:
scheduler.mine.block_tonnage

In [ ]:
for key, row in scheduler.actions_taken.items():
    print(f"{key}: {row['actions_taken']}")

In [ ]:
action

In [ ]:
now = pd.Timestamp.now()
formatted_now = now.strftime("%Y%m%d_%H%M%S")
data = []

In [ ]:
len(arr.flatten())

In [ ]:
with open(f"/home/mustavi/mining/00blocks_ipynb/results/{len(arr.flatten())}_schedular_{formatted_now}_blockscheduler_conversation_log.txt", "w") as f:
    for msg in conversation:
        f.write(f"{msg['role'].upper()}:\n{msg['content']}\n\n")

In [ ]:
for key, val in scheduler.actions_taken.items():
    time_period = key
    print(f"Period: {time_period}",end=' ')
    npv = val['npv'][-1]
    print(f"Scheduler NPV: {npv}")
    data.append((time_period, npv))
npv_df = pd.DataFrame(data, columns=['Period', 'NPV'])
csv_file = f'/home/mustavi/mining/00blocks_ipynb/results/{len(arr.flatten())}_schedular_solution_for_period_{num_periods}_{formatted_now}_different_block.csv'
npv_df.to_csv(csv_file, index=False)
npv_df.plot(x='Period', y='NPV', marker='o', title='NPV over Time Periods')
plt.xlabel('Period')
plt.ylabel('NPV')
plt.grid()
# plt.savefig(f'npv_over_time_{choice}_solution_for_period_{num_periods}_{formatted_now}.png')
plt.show()

In [ ]:
scheduler.mine.npv

In [ ]:
rows = []
for key, triples in scheduler.actions_taken.items():
    for a, b, c in triples['actions_taken']:
        rows.append([key, a, b, c])

df = pd.DataFrame(rows, columns=["Time Period", "Action", "Block", "Amount"])
df.to_csv(f"{csv_file.split('.')[0]}_actions.csv", index=False)

In [ ]:
# print("Scheduler Response:", response['message'])

In [ ]:
# action = (choice,block_number,amount)
# action, pv = scheduler.choosen_action(action)
# scheduler.update_action(action,present_value=pv)
# scheduler.action(action)

In [ ]:
# scheduler.possible_actions()

In [ ]:
# prompt = f"choice was:{choice}, block_number was:{block_number}, amount was:{amount}. Time_period-{period}:Here are the updated possible actions: {scheduler.possible_actions()}"

# conversation.append({"role": "user", "content": prompt})
# gpt_connversation.append({"role": "user", "content": prompt})
# response = ask_ollama(gpt_connversation)
# print(prompt)
# print("Scheduler Response:", response)
# conversation.append({"role": "Scheduler", "content": response['message']})

# # with open("scheduler_conversation_log.txt", "a") as f:
# #     for msg in prompt:
# #         f.write(f"{msg['role'].upper()}:\n{msg['content']}\n\n")

In [ ]:
# with open("scheduler_conversation_log.txt", "w") as f:
#     for msg in conversation:
#         f.write(f"{msg['role'].upper()}:\n{msg['content']}\n\n")

In [ ]:
# assistant_msg = response['message']['content']
# parsed = json.loads(assistant_msg)
# choice = parsed['choice']
# block_number = parsed['block_number']
# amount = parsed['amount']
# print("Choice:", choice)
# print("Block Number:", block_number)
# print("Amount:", amount)
# action = (choice,block_number,amount)
# action, pv = scheduler.choosen_action(action)
# scheduler.update_action(action,present_value=pv)
# scheduler.action(action)

In [ ]:
# scheduler.possible_actions()